# Finding Transformation between Two Cameras 
1. Stretch Camera ( has a known transformation from base frame)
2. Manip/Arm Camera ( unknown from base frame)

In [ ]:
from ai2thor.robot_client import Controller
from ai2thor.robot_grasping import ObjectDetector, GraspPlanner
import matplotlib.pyplot as plt
import cv2
import numpy as np


# initialize 
c = Controller(host='172.16.121.205', port=50051, width=1280, height=720, get_depth=False, multi_thread=False, camera_sources=["arm", "stretch"])

In [ ]:
# rotate stretch camera to face the manipulation workspace
c.step("RotateHead")

# move arm up and extend such that the aruco markers are visible from both camear
c.step({"action":[
                    #{"action": "MoveArmExtension", "args": {"move_scalar": 0.05}},
                    {"action": "MoveArmBase", "args": {"move_scalar": .05}},

]})

# get images
arm_image = c.last_event.third_party_camera_frames[0]
stretch_image = c.last_event.third_party_camera_frames[1]

plt.imshow(arm_image)
plt.show()
plt.imshow(stretch_image)
plt.show()


In [ ]:
# Detect Aruco Marker
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters =  cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

In [ ]:
## ARM 
gray_arm_image = cv2.cvtColor(arm_image, cv2.COLOR_BGR2GRAY)

image_height, image_width = gray_arm_image.shape

arm_aruco_corners, arm_aruco_ids, aruco_rejected_image_points = detector.detectMarkers(gray_arm_image)

print(arm_aruco_corners)
print(arm_aruco_ids)

image=arm_image.copy()
cv2.aruco.drawDetectedMarkers(image, arm_aruco_corners, arm_aruco_ids) 
plt.imshow(image)
plt.show()


## STRETCH
gray_stretch_image = cv2.cvtColor(stretch_image, cv2.COLOR_BGR2GRAY)

image_height, image_width = gray_stretch_image.shape

stretch_aruco_corners, stretch_aruco_ids, aruco_rejected_image_points = detector.detectMarkers(gray_stretch_image)

print(stretch_aruco_corners)
print(stretch_aruco_ids)
#print(aruco_rejected_image_points)

image=stretch_image.copy()
stretch_image_w_aruco = cv2.aruco.drawDetectedMarkers(image, stretch_aruco_corners, stretch_aruco_ids) 
plt.imshow(image)
plt.show()

In [ ]:
# Get Camera Intrinsics for pose estimation
stretch_cam_intr_file = "images/camera_intrinsics/camera_intrinsics_102422073668.txt"
arm_cam_intr_file = "images/camera_intrinsics/camera_intrinsics_234222300666.txt"

import json

with open(stretch_cam_intr_file, 'r') as file:
    # Load the JSON data from the file
    stretch_intr = json.load(file)
    # Print the loaded JSON data
    print(stretch_intr)

with open(arm_cam_intr_file, 'r') as file:
    # Load the JSON data from the file
    arm_intr = json.load(file)
    # Print the loaded JSON data
    print(arm_intr)


In [ ]:
## POSE ESTIMATION
def my_estimatePoseSingleMarkers(corners, marker_size, mtx, distortion):
    '''
    This will estimate the rvec and tvec for each of the marker corners detected by:
       corners, ids, rejectedImgPoints = detector.detectMarkers(image)
    corners - is an array of detected corners for each detected marker in the image
    marker_size - is the size of the detected markers
    mtx - is the camera matrix
    distortion - is the camera distortion matrix
    RETURN list of rvecs, tvecs, and trash (so that it corresponds to the old estimatePoseSingleMarkers())
    '''
    marker_points = np.array([[-marker_size / 2, marker_size / 2, 0],
                              [marker_size / 2, marker_size / 2, 0],
                              [marker_size / 2, -marker_size / 2, 0],
                              [-marker_size / 2, -marker_size / 2, 0]], dtype=np.float32)
    trash = []
    rvecs = []
    tvecs = []
    
    for c in corners:
        nada, R, t = cv2.solvePnP(marker_points, c, mtx, distortion, False, cv2.SOLVEPNP_IPPE_SQUARE)
        rvecs.append(R)
        tvecs.append(t)
        trash.append(nada)
    return rvecs, tvecs, trash


## STRETCH
length_of_marker_mm = 23.5
camera_matrix = np.array([[stretch_intr["fx"], 0, stretch_intr["ppx"]], [0, stretch_intr["fy"], stretch_intr["ppy"]], [0, 0, 1]])
distortion_coefficients = np.array(stretch_intr["coeffs"])
rvecs, tvecs, unknown_variable = my_estimatePoseSingleMarkers(stretch_aruco_corners,
                                                                         length_of_marker_mm,
                                                                         camera_matrix,
                                                                         distortion_coefficients)

stretch_aruco_rotation = rvecs[0]
stretch_aruco_position = tvecs[0]/1000.0 # Convert ArUco position estimate to be in meters.
print(stretch_aruco_rotation)
print(stretch_aruco_position)

P_aruco_from_stretch = np.identity(4)
P_aruco_from_stretch[:3,:3] = cv2.Rodrigues(stretch_aruco_rotation)[0]
P_aruco_from_stretch[0:3,3] = stretch_aruco_position.T
print(P_aruco_from_stretch)


## ARM
length_of_marker_mm = 23.5
camera_matrix = np.array([[arm_intr["fx"], 0, arm_intr["ppx"]], [0, arm_intr["fy"], arm_intr["ppy"]], [0, 0, 1]])
distortion_coefficients = np.array(arm_intr["coeffs"])
rvecs, tvecs, unknown_variable = my_estimatePoseSingleMarkers(arm_aruco_corners,
                                                                         length_of_marker_mm,
                                                                         camera_matrix,
                                                                         distortion_coefficients)

arm_aruco_rotation = rvecs[0]
arm_aruco_position = tvecs[0]/1000.0 # Convert ArUco position estimate to be in meters.
print(arm_aruco_rotation)
print(arm_aruco_position)

P_aruco_from_arm = np.identity(4)
P_aruco_from_arm[:3,:3] = cv2.Rodrigues(arm_aruco_rotation)[0]
P_aruco_from_arm[0:3,3] = arm_aruco_position.T
print(P_aruco_from_arm)


## Useful relationships 

P_object_from_base = T_camera_from_base @ P_object_from_camera


T_stretch_from_arm @ P_aruco_from_stretch   = P_aruco_from_arm

T_arm_from_stretch @ P_aruco_from_arm   = P_aruco_from_stretch


T_arm_from_base =  T_stretch_from_base @ T_arm_from_stretch

==>

T_arm_from_stretch =  P_aruco_from_stretch @ (P_aruco_from_arm)_inverse = P_aruco_from_stretch @ P_arm_from_aruco




#### INVERSE OF HOMOGENEROUS MATRIX 

- R_inv => Transpose of R
- t_inv => -R_Inv * t

In [ ]:
def inverse_homogeneous_matrix(matrix):
    """
    Compute the inverse of a 4x4 homogeneous transformation matrix.

    Args:
    matrix (numpy.ndarray): A 4x4 homogeneous transformation matrix.

    Returns:
    numpy.ndarray: The inverse of the input matrix.
    """
    if matrix.shape != (4, 4):
        raise ValueError("Input matrix must be a 4x4 matrix.")

    rotation_matrix = matrix[0:3, 0:3]
    translation_vector = matrix[0:3, 3]

    inverse_rotation = np.transpose(rotation_matrix)
    inverse_translation = -np.dot(inverse_rotation, translation_vector)

    inverse_matrix = np.identity(4)
    inverse_matrix[0:3, 0:3] = inverse_rotation
    inverse_matrix[0:3, 3] = inverse_translation

    return inverse_matrix


# compute inverse 
P_arm_from_aruco = inverse_homogeneous_matrix(P_aruco_from_arm)
print(P_arm_from_aruco)

# compute transformation matrix 
T_arm_from_stretch =  P_aruco_from_stretch @ P_arm_from_aruco
print(T_arm_from_stretch)

# confirmation
print("--------CONFIRMATION--------")
T_stretch_from_arm = inverse_homogeneous_matrix(T_arm_from_stretch)
recomputed_P_aruco_from_arm =  T_stretch_from_arm @ P_aruco_from_stretch
print(recomputed_P_aruco_from_arm)
print(P_aruco_from_arm)



# compute transfomration matrix for arm from base 
print("--------T_arm_from_base--------")
from scipy.spatial.transform import Rotation as R
r = R.from_quat([0.616, 0.616, -0.346, 0.345]) # camera_color_optical_frame
T_stretch_from_base = np.identity(4)
T_stretch_from_base[0:3, 0:3] = r.as_matrix()
T_stretch_from_base[0:3, 3] = np.array([-0.017, -0.038, 1.294])

T_arm_from_base =  T_stretch_from_base @ T_arm_from_stretch
print(T_arm_from_base)


In [ ]:
T_arm_from_base